<a href="https://colab.research.google.com/github/Itsuki-Hamano123/experiment_search_engine/blob/master/elmo_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 形態素解析ツール用意
https://qiita.com/Fulltea/items/90f6ebe6dcceaf64eaef

In [0]:
# mecabのインストール
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

In [0]:
# Neologd辞書のインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

In [0]:
import MeCab
import subprocess

In [4]:
!ls

drive  ELMoForManyLangs  mecab-ipadic-neologd  sample_data


In [39]:
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')
# Neologd辞書の設定
m_neologd = MeCab.Tagger("-d {0}".format(neologd_path))

m_neologd_sample_txt = "8月3日に放送された「中居正広の金曜日のスマイルたちへ」(TBS系)で、1日たった5分でぽっこりおなかを解消するというダイエット方法を紹介。キンタロー。のダイエットにも密着。"

print("Mecab ipadic NEologd:\n",m_neologd.parse(m_neologd_sample_txt))

ERROR! Session/line number was not unique in database. History logging moved to new session 63
Mecab ipadic NEologd:
 8月3日	名詞,固有名詞,一般,*,*,*,8月3日,ハチガツミッカ,ハチガツミッカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
放送	名詞,サ変接続,*,*,*,*,放送,ホウソウ,ホーソー
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れ	動詞,接尾,*,*,一段,連用形,れる,レ,レ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
「	記号,括弧開,*,*,*,*,「,「,「
中居正広の金曜日のスマイルたちへ	名詞,固有名詞,一般,*,*,*,中居正広の金曜日のスマイルたちへ,ナカイマサヒロノキンヨウビノスマイルタチヘ,ナカイマサヒロノキンヨービノスマイルタチヘ
」(	記号,一般,*,*,*,*,*
TBS	名詞,固有名詞,一般,*,*,*,TBS,ティービーエス,ティービーエス
系	名詞,接尾,一般,*,*,*,系,ケイ,ケイ
)	記号,一般,*,*,*,*,*
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
、	記号,読点,*,*,*,*,、,、,、
1日	名詞,固有名詞,一般,*,*,*,1日,ツイタチ,ツイタチ
たった	副詞,助詞類接続,*,*,*,*,たった,タッタ,タッタ
5分	名詞,固有名詞,一般,*,*,*,5分,ゴフン,ゴフン
で	助詞,格助詞,一般,*,*,*,で,デ,デ
ぽっこりおなか	名詞,固有名詞,一般,*,*,*,ぽっこりおなか,ポッコリオナカ,ポッコリオナカ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
解消	名詞,サ変接続,*,*,*,*,解消,カイショウ,カイショー
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
という	助詞,格助詞,連語,*,*,*,という,トイウ,トユウ
ダイエット方法	名詞,固有名詞,一般,*,*,*,ダイエット方法,ダイエットホウホウ,ダイエットホウホー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
紹介	名詞,サ変接続,*,*,*,*,紹介,ショウカイ,ショーカイ
。	記号,句点,*,*,*,*,。,。,。
キンタ

## ELMoモデルの準備
ELMoForManyLangsを使う

In [0]:
# ELMoForManyLangsに必要なライブラリ
!pip install allennlp

### ELMoForManyLangsを利用
https://github.com/HIT-SCIR/ELMoForManyLangs#use-elmoformanylangs-programmatically

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
cd drive/My Drive/機械学習練習/全文検索エンジン作成

/content/drive/My Drive/機械学習練習/全文検索エンジン作成


In [11]:
!ls

build	 elmoformanylangs	    embedding		  setup.py
configs  elmoformanylangs.egg-info  mecab-ipadic-neologd
dist	 elmo_search_engine.ipynb   README.md


In [0]:
# ELMoForManyLangsのセットアップ
!python setup.py install

In [0]:
import datetime
from pprint import pprint
import pickle
import time
from elmoformanylangs import Embedder

In [0]:
def time_watch(func):
    """時間計測用デコレータ関数
    
    Parameters
    -----
    func : function
        実行したい関数

    """
    import functools
    import datetime
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = datetime.datetime.today()
        result = func(*args, **kwargs)
        end = datetime.datetime.today()
        print("実行時間:{time}".format(time=end - start))
        return result
    return wrapper

### 日本語の事前学習済み重みのダウンロード
embeddingディレクトに格納した<br>
https://qiita.com/mkt3/items/9577b63900109ff91665


In [0]:
# モデルのディレクトリ
MODEL_DIR = './embedding/word_unit_embedding_model/'

In [16]:
elmo_model = Embedder(MODEL_DIR)

2020-03-29 07:33:28,934 INFO: word embedding size: 387179
2020-03-29 07:33:42,783 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(387179, 100, padding_idx=3)
    )
    (projection): Linear(in_features=100, out_features=512, bias=True)
  )
  (encoder): ElmobiLm(
    (forward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (backward_layer_0): LstmCellWithProjection(
      (input_linearity): Linear(in_features=512, out_features=16384, bias=False)
      (state_linearity): Linear(in_features=512, out_features=16384, bias=True)
      (state_projection): Linear(in_features=4096, out_features=512, bias=False)
    )
    (forward_layer_1): LstmCellWithProjection(
      (input_linearity): Line

### 入力文書の前処理を定義

In [0]:
def is_stop_word(pos):
    """
    品詞からストップワードか判断

    Parameters
    -----
    pos : string
        品詞

    Returns
    -----
    stop_word_flag : boolean
    """
    stop_word_list = ["助詞", "助動詞", "記号", "BOS/EOS"]
    stop_word_flag = True
    if pos in stop_word_list:
        stop_word_flag = False
    return stop_word_flag

def split_text(text):
    """
    文章をわかち書きする

    Parameters
    -----
    text : string

    Returns
    -----
    result : list of string
    """
    result = []
    node = m_neologd.parseToNode(text)
    while node:
        if is_stop_word(node.feature.split(",")[0]):
            result.append(node.surface)
        node = node.next
    return result

In [0]:
def preprocessing_documents(documents):
    """
    文書をELMo用に前処理

    Parameters
    -----
    documents : list of string

    Returns
    -----
    results : list of list of string
    """
    results = []
    # バグ対処
    m_neologd.parse('')
    for document in documents:
        result = split_text(document)
        results.append(result)
    return results

In [0]:
elmo_sample_docments = ["私は犬", "犬は私", "今日の天気は雪のち晴れです"]

In [73]:
proc_documents = preprocessing_documents(elmo_sample_docments)
proc_documents

[['私', '犬'], ['犬', '私'], ['今日の天気', '雪', 'のち', '晴れ']]

### ELMoで単語の分散表現を予測

In [0]:
@time_watch
def predict_word_vec(model, proc_documents):
    """
    モデルを使い単語の分散表現を予測

    Parmeters
    -----
    model : model_obj
    proc_documents : proc_documents_obj
    
    Returns
    -----
    pred_vector : list of nparray
    """
    pred_vector = model.sents2elmo(proc_documents)
    return pred_vector

In [75]:
documnets_vec = predict_word_vec(elmo_model, proc_documents)

2020-03-29 08:48:47,695 INFO: 1 batches, avg len: 4.7


実行時間:0:00:00.359381


In [76]:
documnets_vec

[array([[-0.3822073 , -0.39298368, -0.19943719, ..., -0.0947518 ,
          0.38196918, -0.06013573],
        [-0.99855757, -0.7653184 , -0.67188597, ...,  0.38484803,
         -0.14899619, -0.22106065]], dtype=float32),
 array([[-0.85666806, -0.24317439, -0.31542605, ..., -0.16575427,
          0.06315442, -0.31759426],
        [-0.29320082, -0.47321966, -0.83421296, ..., -0.40913573,
          0.21732388,  0.24414034]], dtype=float32),
 array([[ 0.10117573, -0.23624979,  0.00397639, ..., -0.2328723 ,
          0.17794509,  0.0607481 ],
        [-0.90098685, -0.80577874, -0.20233335, ...,  0.13288653,
          0.4778336 , -0.45927873],
        [ 1.2292985 ,  0.98217875, -0.351058  , ...,  0.05930225,
          0.5154347 , -0.4115046 ],
        [-0.65625954,  1.4385008 , -0.17021495, ..., -0.32545912,
          0.10576967,  0.18586273]], dtype=float32)]

#### 実行結果
「私は犬」と「犬は私」の２つの文章において、それぞれの文章で「私」「犬」は文章内での順番が考慮され、別々の分散表現が得られていることが分かる。